In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ["MLFLOW_TRACKING_URL"] = "https://dagshub.com/subhadip18/datascienceproject.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "subhadip18"
MLFLOW_TRACKING_PASSWORD = os.getenv("MLFLOW_TRACKING_PASSWORD")

In [4]:
os.chdir("../")  # Change directory to project root for mlflow tracking
%pwd

'd:\\CampusX\\Miscellaneous topics\\datascienceproject'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_url: str

In [7]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_url="https://dagshub.com/subhadip18/datascienceproject.mlflow"
        )

        return model_evaluation_config


In [9]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [15]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]

        mlflow.set_registry_uri(self.config.mlflow_url)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

            ## Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")


In [16]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-12-04 18:00:34,763: INFO: common]: yaml file: config\config.yaml loaded successfully
[2025-12-04 18:00:34,766: INFO: common]: yaml file: params.yaml loaded successfully
[2025-12-04 18:00:34,766: INFO: common]: yaml file: params.yaml loaded successfully
[2025-12-04 18:00:34,803: INFO: common]: yaml file: schema.yaml loaded successfully
[2025-12-04 18:00:34,806: INFO: common]: created directory at: artifacts_root
[2025-12-04 18:00:34,808: INFO: common]: created directory at: artifacts/model_evaluation
[2025-12-04 18:00:34,803: INFO: common]: yaml file: schema.yaml loaded successfully
[2025-12-04 18:00:34,806: INFO: common]: created directory at: artifacts_root
[2025-12-04 18:00:34,808: INFO: common]: created directory at: artifacts/model_evaluation
[2025-12-04 18:01:59,067: INFO: common]: json file saved at: artifacts\model_evaluation\metrics.json
[2025-12-04 18:01:59,067: INFO: common]: json file saved at: artifacts\model_evaluation\metrics.json


2025/12/04 18:03:43 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/12/04 18:03:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/12/04 18:03:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
